# Clustering Crypto

In [83]:
# Initial imports
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [84]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [85]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

#data = requests.get(url).json()
#data


In [86]:
# Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")

# Create a DataFrame

crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [87]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()




,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [88]:
# Keep only cryptocurrencies that are trading

crypto_df = crypto_df[crypto_df["IsTrading"]==True]
crypto_df.head()



,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [89]:
# Keep only cryptocurrencies with a working algorithm

# Not sure how to determine a working algorithm from dataset provided. 


In [90]:
# Remove the "IsTrading" column

crypto_df.drop(['IsTrading'], axis = 1, inplace = True)
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [91]:
# Remove rows with at least 1 null value

crypto_df = crypto_df[pd.notnull(crypto_df["TotalCoinsMined"])]
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [92]:
# Remove rows with cryptocurrencies having no coins mined

crypto_df.drop(crypto_df.loc[crypto_df["TotalCoinsMined"]==0.000000e+00].index, inplace=True)
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [93]:
# Drop rows where there are 'N/A' text values

crypto_df = crypto_df.dropna().copy() 


In [94]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

coin_namedf = pd.DataFrame(crypto_df["CoinName"])
coin_namedf.head()



,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [95]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm

crypto_df.drop(['CoinName'], axis = 1, inplace = True)
crypto_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [96]:
# Create dummy variables for text features

X = pd.get_dummies(crypto_df)
X.head()


,TotalCoinsMined,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
42,4.199995e+01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,1.055185e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
# Standardize data

ss_crypto_df = StandardScaler().fit_transform(X)
ss_crypto_df


array([[-0.11674788, -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231, -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [98]:
# Use PCA to reduce dimensions to 3 principal components

pca = PCA(n_components=3)
cryptodf_pca = pca.fit_transform(ss_crypto_df)



In [99]:
# Create a DataFrame with the principal components data

pcs_df = pd.DataFrame(data = cryptodf_pca, columns = ["PC 1", "PC 2", "PC 3"], index = crypto_df.index)
pcs_df.head()



,PC 1,PC 2,PC 3
42,-0.319884,0.031790,-0.000254
404,-0.305927,0.038662,-0.004613
1337,0.402812,0.083740,0.042561
BTC,-0.225843,-0.101184,-0.068309
ETH,-0.169229,-0.065328,-0.104012


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [100]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for possible_k in k:
    km_model = KMeans(n_clusters=possible_k, random_state=5)
    km_model.fit(pcs_df)
    inertia.append(km_model.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [101]:
# Initialize the K-Means model
#def get_clusters(k, pcs_df):
model = KMeans(n_clusters=3, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"] = model.labels_
    
    #return pcs_df

In [102]:
# Create a new DataFrame named clustered_df, that includes the following columns "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class". You should maintain the index of the crypto_df

clustered_df = pd.concat([crypto_df, coin_namedf, pcs_df], axis="columns", sort=False)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC 1,PC 2,PC 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.319884,0.031790,-0.000254,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.305927,0.038662,-0.004613,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,0.402812,0.083740,0.042561,0
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.225843,-0.101184,-0.068309,0
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.169229,-0.065328,-0.104012,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [103]:
# Create a 3D-Scatter with the PCA data and the clusters


fig = px.scatter_3d(clustered_df, 
                    x='PC 1', 
                    y='PC 2', 
                    z='PC 3',
                    color='Algorithm',
                    symbol='class', 
                    hover_name = "CoinName",
                    hover_data = ["Algorithm"],
                    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


#### Table of Tradable Cryptocurrencies

In [104]:
# Table with tradable cryptos

clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [105]:
# Print the total number of tradable cryptocurrencies

clustered_df.shape[0]


533

#### Scatter Plot with Tradable Cryptocurrencies

In [106]:
# Scale data to create the scatter plot

mmscaler = MinMaxScaler()
data = mmscaler.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
data_df = pd.DataFrame(data, columns = ["TotalCoinSupply", "TotalCoinsMined"])
data_df["CoinName"] = clustered_df["CoinName"]
data_df["class"] = clustered_df["class"]


In [115]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

clustered_df.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    hover_cols=["CoinName"],
)




:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)